In [ ]:
#| default_exp ai

# 🧠 AI-Powered Text Operations

Multi-provider AI operations using the `lisette` library for flexible text editing and analysis.

**Features:**
- Multi-model support (Gemini, Claude, OpenAI, etc.)
- Natural language instructions for text editing
- Web search capabilities for real-time information
- Transcript summarization and improvement
- Change explanation and analysis
- Efficient conversation management (single assistant message + cumulative user instructions)
- Natural language support for commands like "change it back" or "undo that"

In [ ]:
#| export
from typing import List, Dict, Literal, Union
from pydantic import BaseModel, ConfigDict, Field, model_validator
from dotenv import load_dotenv
import re
import json
from lisette import Chat


load_dotenv()

True

## 📦 Core AI Functions

AI-powered operations using lisette's flexible multi-provider interface.

**Main Functions:**
- `ai_chat()` — General-purpose AI chat with multi-model support
- `summarize_transcript()` — Generate concise summaries
- `explain_edits()` — Natural language explanation of changes
- `improve_transcript()` — Flexible text improvement with custom instructions

In [ ]:
#| export

# --- Replace all ------------------------------------------------------------

class ReplaceAllOp(BaseModel):
    """Represents a 'replace all' text operation."""
    op: Literal["replace_all"]
    find: str = Field(..., min_length=1)
    replace: str = Field(..., min_length=0)
    model_config = ConfigDict(extra="forbid")


# --- Regex replace ------------------------------------------------------------

class RegexReplaceOp(BaseModel):
    """Represents a regex-based find/replace operation."""
    op: Literal["regex_replace"]
    pattern: str = Field(..., min_length=1)
    replacement: str = Field(..., min_length=0)
    model_config = ConfigDict(extra="forbid")

    @model_validator(mode="after")
    def _validate_regex(cls, v: "RegexReplaceOp"):
        # Precompile regex to ensure it's valid
        try:
            re.compile(v.pattern)
        except re.error as e:
            raise ValueError(f"Invalid regex pattern: {e}") from e
        return v

# --- Insert at absolute position ---------------------------------------------

class InsertAtOp(BaseModel):
    """Insert text at an absolute character position (0-indexed)."""
    op: Literal["insert_at"]
    text: str = Field(..., min_length=1)
    position: int = Field(..., ge=0)
    model_config = ConfigDict(extra="forbid")

# --- Insert after marker ------------------------------------------------------

class InsertAfterOp(BaseModel):
    """Insert text after the first occurrence of a marker string."""
    op: Literal["insert_after"]
    text: str = Field(..., min_length=1)
    after: str = Field(..., min_length=1)
    model_config = ConfigDict(extra="forbid")

# --- Delete -------------------------------------------------------------------

class DeleteOp(BaseModel):
    """Delete exact text (first or all occurrences)."""
    op: Literal["delete"]
    text: str = Field(..., min_length=1)
    all_occurrences: bool = False
    model_config = ConfigDict(extra="forbid")


# --- Edit plan container ------------------------------------------------------

class EditPlan(BaseModel):
    """Represents a list of text operations to apply sequentially."""
    ops: List[
        Union[
            ReplaceAllOp,
            RegexReplaceOp,
            InsertAtOp,
            InsertAfterOp,
            DeleteOp,
        ]
    ]
    model_config = ConfigDict(extra="forbid")

## 🧰 Conversation Management

The AI conversation uses a hybrid context pattern for efficiency.

**Session State:**
- `_messages` — conversation history
- `_current` — current transcript after applied edits

**Structure:**
- **System message:** defines AI role and available operations
- **Assistant message:** contains current transcript (updated after each edit)
- **User messages:** cumulative instruction history

**Example after 2 edits:**
```json
[
  {"role": "system", "content": "You are a precise text editor..."},
  {"role": "assistant", "content": "Here is the current transcript:\nI met oscar on Monday."},
  {"role": "user", "content": "Instruction: Change him to oscar"},
  {"role": "user", "content": "Instruction: Change yesterday to on Monday"}
]
```

**Functions:**
- `_new_conversation(transcript)` — initializes conversation with system and assistant messages
- `_set_current_transcript(new_transcript)` — updates assistant message with latest transcript

In [ ]:
# | export
# --- session state (module-level) ---
_messages: List[Dict[str, str]] | None = None
_current: str | None = None

def _new_conversation(transcript: str) -> List[Dict[str, str]]:
    """Create a new message list with system + assistant context."""
    return [
        {
            "role": "system",
            "content": (
                "You are a precise text editor that outputs ONLY valid JSON matching the EditPlan schema.\n\n"
                "Available operations:\n"
                "1. replace_all — exact literal text only (no regex)\n"
                "   fields:\n"
                "       - find: the exact text to replace\n"
                "       - replace: replacement text for every occurrence\n\n"
                "2. regex_replace - pattern-based replacements (e.g., dates)\n"
                "   fields:\n"
                "       - pattern: regex pattern to match (e.g., (\\d{4})-(\\d{2})-(\\d{2}) for dates)\n"
                "       - replacement: replacement string using \\1, \\2 for capture groups\n\n"
                "3. insert_at — insert text at an absolute index (0 = start)\n"
                "   fields:\n"
                "       - text: text to insert\n"
                "       - position: integer index to insert at\n\n"
                "4. insert_after — insert text after a marker\n"
                "   fields:\n"
                "       - text: text to insert\n"
                "       - after: insert after the first occurrence of this string\n"
                "       (ALWAYS provide a space in the string if needed when doing insert)\n\n"
                "5. delete — remove exact text\n"
                "   fields:\n"
                "       - text: the exact text to remove\n"
                "       - all_occurrences: true = remove all, false = only first (default false)\n\n"
                "RULES:\n"
                "- If you see regex patterns or date formats, you MUST use regex_replace, NOT replace_all!\n"
                "- When interpreting natural or spoken language, infer the user's intent precisely and map it to the correct fields.\n"
                "- ALWAYS provide a space in text to insert if needed.\n"
                "- Respond ONLY with valid JSON following the EditPlan schema."
            ),
        },
        {
            "role": "assistant",
            "content": f"Current text to edit:\n{transcript}",
        },
    ]


def _set_current_transcript(new_transcript: str) -> None:
    global _messages
    # replace the single assistant transcript message
    for m in _messages:
        if m.get("role") == "assistant":
            m["content"] = f"Current text to edit:\n{new_transcript}"
            return
    # Fallback: insert one if missing
    _messages.insert(1, {
        "role": "assistant",
        "content": f"Current text to edit:\n{new_transcript}",
    })

## 🤖 Core Functions

**`_plan_edits(instruction, model)`**
- Appends user instruction to conversation
- Calls LLM with `response_format=EditPlan` for structured output
- Returns parsed `EditPlan` object

**`_apply_plan(transcript, plan)`**
- Applies all operations in `EditPlan` sequentially to the transcript
- Supports: `replace_all`, `regex_replace`, `insert_at`, `insert_after`, `delete`
- Returns updated transcript

In [ ]:
# | export

def _plan_edits(instruction: str, model: str = "gemini/gemini-2.5-flash") -> EditPlan:
    """
    Append a user instruction, call the model with structured output, and return the parsed plan.
    """
    global _messages

    # Add the new instruction to the conversation
    _messages.append({"role": "user", "content": f"Instruction: {instruction}"})

    # Use lisette to get structured JSON response
    chat = Chat(model, response_format="json")
    
    # Format messages for lisette (convert our format to lisette's expected format)
    response = chat(messages=_messages, temperature=0)
    
    # Extract the JSON content from response
    content = response.choices[0].message.content
    
    # Parse JSON and validate with Pydantic
    try:
        data = json.loads(content)
        plan = EditPlan.model_validate(data)
        return plan
    except (json.JSONDecodeError, Exception) as e:
        raise RuntimeError(f"Failed to parse model response as EditPlan: {e}\nResponse: {content}")


def _apply_plan(transcript: str, plan: EditPlan) -> str:
    """
    Apply all operations from the EditPlan to the given transcript.
    """
    updated = transcript
    for op in plan.ops:
        if op.op == "replace_all":
            updated = updated.replace(op.find, op.replace)
        elif op.op == "regex_replace":
            updated = re.sub(op.pattern, op.replacement, updated)
        elif op.op == "insert_at":
            pos = max(0, min(op.position, len(updated)))
            updated = updated[:pos] + op.text + updated[pos:]
        elif op.op == "insert_after":
            idx = updated.find(op.after)
            if idx != -1:
                insert_pos = idx + len(op.after)
                updated = updated[:insert_pos] + op.text + updated[insert_pos:]
        elif op.op == "delete":
            if op.all_occurrences:
                updated = updated.replace(op.text, "")
            else:
                # Delete first occurrence only
                idx = updated.find(op.text)
                if idx != -1:
                    updated = updated[:idx] + updated[idx + len(op.text):]
    return updated

## 🔌 Public API

Functions for managing edit sessions and applying instructions.

In [ ]:
# | export
def has_session() -> bool:
    """Return True if an edit session is initialized."""
    return _messages is not None and _current is not None

def start_session(initial_transcript: str) -> str:
    """Seed a new session with the initial transcript and return it."""
    global _messages, _current
    _current = initial_transcript
    _messages = _new_conversation(initial_transcript)
    return _current

def apply_instruction(instruction: str, model: str = "gemini/gemini-2.5-flash") -> str:
    """Apply an instruction to the current transcript and return the updated text."""
    global _current
    if not has_session():
        raise RuntimeError("No session. Call start_session() first.")
    plan = _plan_edits(instruction, model)
    _current = _apply_plan(_current, plan)
    _set_current_transcript(_current)
    return _current

def current_transcript() -> str:
    """Get the latest edited transcript (or '' if none)."""
    return _current or ""

def reset_session() -> None:
    """Clear session state."""
    global _messages, _current
    _messages, _current = None, None

## 🌟 Lisette Integration (Multi-Model AI)

Additional AI capabilities using the `lisette` library for flexible, multi-provider AI operations.

**Use Cases:**
- General chat and Q&A with multiple AI providers (Gemini, Claude, OpenRouter models, etc.)
- Web search-enabled queries
- Transcript summarization and analysis
- Explaining edits between versions

**Note:** All AI operations in this module use lisette for consistent multi-provider support.

In [ ]:
# | export

def ai_chat(
    prompt: str, 
    model: str = "gemini/gemini-2.5-flash", 
    enable_search: bool = False
) -> str:
    """
    General-purpose AI chat using lisette for multi-provider support.
    
    Args:
        prompt: Question or instruction for the AI
        model: Model identifier (e.g., "gemini/gemini-2.5-flash", "claude-sonnet-4-20250514", "gpt-4o")
        enable_search: Whether to enable web search capabilities
    
    Returns:
        AI response text
        
    Example:
        >>> response = ai_chat("What is the capital of Norway?", enable_search=True)
        >>> print(response)
    """
    search_level = "l" if enable_search else None
    chat = Chat(model, search=search_level)
    response = chat(prompt)
    return response.choices[0].message.content


def summarize_transcript(
    transcript: str, 
    model: str = "gemini/gemini-2.5-flash",
    max_words: int = 100
) -> str:
    """
    Generate a concise summary of a transcript.
    
    Args:
        transcript: The text to summarize
        model: AI model to use
        max_words: Maximum words for the summary
        
    Returns:
        Summary text
    """
    prompt = f"""Summarize this transcript in {max_words} words or less:

{transcript}

Provide a clear, concise summary."""
    return ai_chat(prompt, model)


def explain_edits(original: str, edited: str, model: str = "gemini/gemini-2.5-flash") -> str:
    """
    Get an AI explanation of what changed between two text versions.
    
    Args:
        original: Original text
        edited: Edited/modified text
        model: AI model to use
        
    Returns:
        Natural language explanation of changes
    """
    prompt = f"""Compare these two versions and explain what changed:

ORIGINAL:
{original}

EDITED:
{edited}

Provide a brief, clear explanation of the changes made."""
    return ai_chat(prompt, model)


def improve_transcript(
    transcript: str,
    instructions: str = "Fix grammar, punctuation, and clarity while preserving meaning",
    model: str = "gemini/gemini-2.5-flash"
) -> str:
    """
    Use AI to improve transcript quality with flexible instructions.
    
    Args:
        transcript: Text to improve
        instructions: How to improve it (grammar, clarity, formality, etc.)
        model: AI model to use
        
    Returns:
        Improved transcript text
    """
    prompt = f"""{instructions}

TEXT:
{transcript}

Return ONLY the improved text, no explanations."""
    return ai_chat(prompt, model)

## 🧪 Lisette Examples

Practical examples showing how to use the lisette-powered functions for various AI tasks.

In [ ]:
#| eval: false
### Example 1: Basic AI Chat

# Simple question without web search
response = ai_chat("What is Python's primary use case?")
print("🤖 AI:", response)

# Question with web search enabled
response = ai_chat("What is the weather in Oslo today?", enable_search=True)
print("🌐 AI with search:", response)

🤖 AI: Python is a highly versatile, general-purpose programming language, so it doesn't have *one single* primary use case in the way a specialized language might. However, if we were to identify its most prominent and impactful areas where it truly shines and dominates, they would be:

1.  **Data Science, Machine Learning, and Artificial Intelligence (AI):** This is arguably Python's strongest and most defining primary use case today.
    *   **Why:** Its rich ecosystem of powerful libraries like NumPy (numerical computing), Pandas (data manipulation and analysis), SciPy (scientific computing), Matplotlib/Seaborn (data visualization), Scikit-learn (machine learning), TensorFlow, and PyTorch (deep learning) makes it the go-to language for data scientists, analysts, and AI researchers.
    *   **What it's used for:** Data cleaning, analysis, visualization, building predictive models, developing AI algorithms, natural language processing, computer vision, and more.

2.  **Web Development

In [ ]:
#| eval: false
### Example 2: Summarize a Transcript

long_transcript = """
I went to the store yesterday and bought some groceries. I got milk, bread, eggs, and cheese.
Then I went to the hardware store to pick up some nails and a hammer. After that, I stopped
by the pharmacy to get my prescription. It was a pretty productive day overall. I also met
my friend Sarah at the coffee shop and we talked for about an hour about her new job.
"""

summary = summarize_transcript(long_transcript, max_words=30)
print("📝 Summary:", summary)

📝 Summary: The speaker had a productive day, buying groceries, hardware, and picking up a prescription. They also met a friend at a coffee shop to discuss her new job.


In [ ]:
#| eval: false
### Example 3: Explain Changes Between Versions

original = "I met him yesterday at the store."
edited = "I met oscar on Monday at the grocery store."

explanation = explain_edits(original, edited)
print("📊 Changes:", explanation)

📊 Changes: The edited version makes the sentence more specific by providing more detailed information:

*   **Person:** "him" was changed to the specific name "Oscar."
*   **Time:** "yesterday" was changed to the specific day "on Monday."
*   **Place:** "the store" was changed to the more specific "the grocery store."


In [ ]:
#| eval: false
### Example 4: Improve Transcript Quality

messy_transcript = "um like i was saying uh the meeting was you know really good and stuff"

improved = improve_transcript(
    messy_transcript,
    instructions="Remove filler words and improve clarity while keeping it casual"
)
print("Original:", messy_transcript)
print("Improved:", improved)

Original: um like i was saying uh the meeting was you know really good and stuff
Improved: The meeting was really good.


## 🔄 Complete Workflow: Lisette-Powered

Demonstrating a complete workflow using **only lisette** for all AI operations - editing, explanation, summarization, and improvement.

In [ ]:
#| eval: false
### Complete Workflow: Using Lisette for Everything

# 1️⃣ Start with a raw transcript
raw_transcript = "I met him yesterday and he told me about the project deadline."
print("📝 Original:", raw_transcript)

# 2️⃣ Use lisette to make edits (flexible AI-based editing)
edit_instruction = "Change 'him' to 'John' and 'yesterday' to 'last Tuesday'. Return only the edited text."
edited = ai_chat(edit_instruction + f"\n\nText: {raw_transcript}")
print("✏️  Edited:", edited)

# 3️⃣ Get AI explanation of changes (lisette)
explanation = explain_edits(raw_transcript, edited)
print("📊 Changes:", explanation)

# 4️⃣ Generate summary (lisette)
summary = summarize_transcript(edited, max_words=15)
print("📄 Summary:", summary)

# 5️⃣ Improve transcript quality (lisette)
improved = improve_transcript(edited, instructions="Make it more formal and professional")
print("✨ Improved:", improved)

📝 Original: I met him yesterday and he told me about the project deadline.
✏️  Edited: I met John last Tuesday and he told me about the project deadline.
📊 Changes: The changes made are:

1.  **"him" was replaced with "John"**: This changes a pronoun to a specific proper noun, making the person's identity clear.
2.  **"yesterday" was replaced with "last Tuesday"**: This changes a relative time reference to a specific day, making the timing of the meeting more precise.
📄 Summary: John informed me about the project deadline.
✨ Improved: I met with John last Tuesday, and he informed me of the project deadline.


## 🧪 Example: Sequential Editing with Lisette

Demonstrates two editing steps using lisette:
1. Replace all "him" with "oscar"
2. Replace "yesterday" with "on Monday"

Each step uses natural language instructions with lisette's flexible AI.

In [ ]:
#| eval: false
### Example: Sequential Editing with Lisette

# Initial transcript
input_text = "I told him that I saw him yesterday. Then I asked him if he could help."
print("📝 Original:", input_text)

# 1️⃣ First edit
instruction1 = "Change all occurrences of 'him' to 'oscar'. Return only the edited text."
input_text = ai_chat(instruction1 + f"\n\nText: {input_text}")
print("✏️  After edit 1:", input_text)

# 2️⃣ Second edit
instruction2 = "Now change 'yesterday' to 'on Monday'. Return only the edited text."
input_text = ai_chat(instruction2 + f"\n\nText: {input_text}")
print("✏️  After edit 2:", input_text)

print("✅ Final transcript:", input_text)

📝 Original: I told him that I saw him yesterday. Then I asked him if he could help.
✏️  After edit 1: I told oscar that I saw oscar yesterday. Then I asked oscar if he could help.
✏️  After edit 2: I told oscar that I saw oscar on Monday. Then I asked oscar if he could help.
✅ Final transcript: I told oscar that I saw oscar on Monday. Then I asked oscar if he could help.


## 🧩 Inspecting the Conversation

Print the `_messages` list to see what the model sees on each call.

**Key observations:**
- One assistant message with the current transcript
- Multiple user instructions recording session history

In [ ]:
#| eval: false
from pprint import pprint

print("🧩 Message history:")
pprint(_messages)


🧩 Message history:
None


## ✅ Summary

**Architecture:**
- Hybrid context: single assistant message (current state) + cumulative user instructions (history)
- Efficient for long transcripts with complex edit sequences
- Supports natural, conversational editing patterns

**Supported Operations:**
1. `replace_all` — exact text replacement
2. `regex_replace` — pattern-based with capture groups (\1, \2, etc.)
3. `insert_at` — insert at character position (0-indexed)
4. `insert_after` — insert after marker string
5. `delete` — remove first or all occurrences

**Future Enhancements:**
- Token usage tracking
- Operation history with undo/redo
- UI integration (TUI/web)

## 🧪 Testing Different Edit Types with Lisette

Let's test various text editing operations using lisette's natural language interface.

In [ ]:
#| eval: false
### Test 1: Date Format Conversion

test_text = "Meeting on 2025-10-07 and another on 2025-12-25."
print(f"Original: {test_text}")

# Use lisette to convert dates
instruction = "Convert all dates from YYYY-MM-DD format to MM/DD/YYYY format. Return only the edited text."
result = ai_chat(instruction + f"\n\nText: {test_text}")
print(f"Result:   {result}")

Original: Meeting on 2025-10-07 and another on 2025-12-25.

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



RateLimitError: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10\nPlease retry in 7.812891951s.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.QuotaFailure",
        "violations": [
          {
            "quotaMetric": "generativelanguage.googleapis.com/generate_content_free_tier_requests",
            "quotaId": "GenerateRequestsPerMinutePerProjectPerModel-FreeTier",
            "quotaDimensions": {
              "location": "global",
              "model": "gemini-2.5-flash"
            },
            "quotaValue": "10"
          }
        ]
      },
      {
        "@type"

In [ ]:
#| eval: false
### Test 2: Insert Space

test_text = "HelloWorld"
print(f"Original: {test_text}")

instruction = "Add a space between Hello and World. Return only the edited text."
result = ai_chat(instruction + f"\n\nText: {test_text}")
print(f"Result:   {result}")

Original: HelloWorld
Result:   Hello World
Result:   Hello World


In [ ]:
#| eval: false
### Test 3: Insert After Marker

test_text = "Hello, my name is John. I love coding."
print(f"Original: {test_text}")

instruction = "Add ' Smith' right after 'John'. Return only the edited text."
result = ai_chat(instruction + f"\n\nText: {test_text}")
print(f"Result:   {result}")

Original: Hello, my name is John. I love coding.
Result:   Hello, my name is John Smith. I love coding.
Result:   Hello, my name is John Smith. I love coding.


In [ ]:
#| eval: false
### Test 4: Delete First Occurrence

test_text = "I like apples and apples are great!"
print(f"Original: {test_text}")

instruction = "Delete only the first occurrence of 'apples'. Return only the edited text."
result = ai_chat(instruction + f"\n\nText: {test_text}")
print(f"Result:   {result}")

Original: I like apples and apples are great!
Result:   I like and apples are great!
Result:   I like and apples are great!


In [ ]:
#| eval: false
### Test 5: Delete All Occurrences

test_text = "I like apples and apples are great!"
print(f"Original: {test_text}")

instruction = "Delete all occurrences of 'apples'. Return only the edited text."
result = ai_chat(instruction + f"\n\nText: {test_text}")
print(f"Result:   {result}")

Original: I like apples and apples are great!
Result:   I like and are great!
Result:   I like and are great!


In [ ]:
#| eval: false
### Test 6: Complex Multi-Operation Edit

test_text = "The meeting is on 2025-10-07 at the office. Please confirm."
print(f"Original: {test_text}")

instruction = """Make these changes:
1. Change the date format from YYYY-MM-DD to MM/DD/YYYY
2. Change 'office' to 'conference room'
3. Add ' (urgent)' at the end

Return only the edited text."""
result = ai_chat(instruction + f"\n\nText: {test_text}")
print(f"Result:   {result}")

Original: The meeting is on 2025-10-07 at the office. Please confirm.
Result:   The meeting is on 10/07/2025 at the conference room. Please confirm. (urgent)
Result:   The meeting is on 10/07/2025 at the conference room. Please confirm. (urgent)


In [ ]:
#from smolagents import CodeAgent, InferenceClientModel, WebSearchTool
#https://huggingface.co/docs/smolagents/index


# Connect to running vLLM server using OpenAI-compatible API
# model = OpenAIServerModel(
#     model_id="Qwen/Qwen3-4B-Instruct-2507",
#     api_base="http://localhost:8000/v1",
#     api_key="dummy",  # vLLM doesn't require a real API key
#     temperature=0.1,  # Lower temperature for more consistent output
# )

# model = InferenceClientModel()
# agent = CodeAgent(
#     tools=[WebSearchTool()],
#     model = model 
# )

# # Test with a simple question first
# print("Testing simple question...")
# response = agent.run("What is the square root of 75?")
# print(f"Response: {response}")